<a href="https://colab.research.google.com/github/YaninaK/anomaly-detection/blob/b1/notebooks/02_Anomalies_1_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение аномалий в начислениях за тепловую энергию


## 2. Аномалии 1, 2 и 4.



### Примеры аномалий

Виды аномалий по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):

1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);

2. равные значения показаний в течение нескольких расчетных периодов;

3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);

4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).



In [1]:
initiate = True
if initiate:
  !git init -q
  !git clone -b b1  https://github.com/YaninaK/anomaly-detection.git -q

  from google.colab import drive
  drive.mount('/content/drive')

  !unzip -u -q /content/drive/MyDrive/ML_projects/08_anomaly_detection/data/01_raw/task#3.zip -d /content/anomaly-detection/data/01_raw

%cd /content/anomaly-detection/notebooks

Mounted at /content/drive
/content/anomaly-detection/notebooks


In [2]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", "src", "anomaly_detection"))

In [3]:
import datetime

import numpy as np
import pandas as pd

from data.make_dataset import load_data
from data.preprocess import Preprocess
from features.data_sequence import generate_data_sequence
from features.missing_records import select_missing_records, select_uninvoiced_objects
from features.duplicated import get_equal_values
from features.grouping import Grouping
from features.period_outliers import get_outlers

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
PATH = "/content/anomaly-detection/"

## 1. Чтение данных

In [6]:
folder_path = '../data/01_raw/'

In [7]:
regenerate = True
data, temperature, buildings = load_data(folder_path, regenerate, path=PATH)

100%|██████████| 24/24 [00:13<00:00,  1.81it/s]


## 2. Подготовка данных

In [8]:
preprocess = Preprocess()
data, buildings = preprocess.fit_transform(data, buildings)

data.shape

(75385, 10)

In [9]:
df = generate_data_sequence(data)
df.shape

print(f"df.shape = {df.shape}\n")
df.sample(2)

df.shape = (4627, 24)



,,,,,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,...,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01
Адрес объекта,Тип объекта,№ ОДПУ,Адрес объекта 2,Вид энерг-а ГВС,,,,,,,,,,,,,,,,,,,,,
"г Уфа, ул. Маршала Жукова, д.2 корп.6",Многоквартирный дом,106657,"г Уфа, ул. Маршала Жукова, д.2 корп.6",NaN,NaN,NaN,NaN,21.770,36.863,48.952,61.149,46.467,48.605,28.346,...,0.000,37.690,38.649,55.2518,56.732,49.695229,35.964,25.878,NaN,NaN
"г Уфа, пр-кт Октября, д.166",Многоквартирный дом,17052549,"г Уфа, пр-кт Октября, д.166",ГВС-ИТП,6.358,3.918,6.089,32.619,41.254,50.666,63.929,49.047,57.530,47.194,...,8.659,29.125,47.426,67.3030,72.054,62.087000,51.337,42.886,7.121,5.835


## 3. Аномалии



### 3.1 Нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель)

#### 3.1.1 Пропуски в данных по текущему потреблению

In [10]:
save=True
all_periods = False
missing_records = select_missing_records(df, all_periods, save, path=PATH)

print(f"missing_records.shape = {missing_records.shape}\n")
missing_records.sample(2)

missing_records.shape = (1052, 14)



,,,,,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00
Адрес объекта,Тип объекта,№ ОДПУ,Адрес объекта 2,Вид энерг-а ГВС,,,,,,,,,,,,,,
"г Уфа, ул. Джалиля Киекбаева, д.17",Многоквартирный дом,800356,"г Уфа, ул. Джалиля Киекбаева, д.17",NaN,123.802,115.024,203.428,217.948,216.866,196.607,201.511,144.627,180.784,331.012,366.65,231.192,235.788,0.00
"г Уфа, ул. Орджоникидзе, д.4",Другое строение,4429866,"г Уфа, ул. Орджоникидзе, д.4",NaN,14.710,NaN,NaN,28.760,24.046,21.020,16.935,13.729,0.000,22.618,31.90,25.670,21.700,16.44


#### 3.1.2 Объекты без данных по текущему потреблению

In [11]:
save = True

uninvoiced_objects = select_uninvoiced_objects(df, buildings, save, path=PATH)

print(f"uninvoiced_objects.shape = {uninvoiced_objects.shape}")
uninvoiced_objects.sample(2)

uninvoiced_objects.shape = (1650, 6)


,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
497,"г Уфа, ул. Индустриальное шоссе, д.26",Другое строение,5,1975-01-01,0.01,"г Уфа, ул. Индустриальное шоссе, д.26"
1349,"г Уфа, ул. Свободы, д.59 корп.3","Административные здания, конторы",2,NaT,0.02,"г Уфа, ул. Свободы, д.59 корп.3"


In [12]:
pd.concat(
    [
        uninvoiced_objects["Тип Объекта"].value_counts(),
        uninvoiced_objects["Тип Объекта"].value_counts(normalize=True)
    ], axis=1
).head(10)

,count,proportion
Тип Объекта,,
Многоквартирный дом,752,0.455758
Другое строение,652,0.395152
Частный дом,138,0.083636
"Административные здания, конторы",45,0.027273
"Учебное заведение, комбинат, центр",13,0.007879
Школы и ВУЗ,13,0.007879
Гаражи,6,0.003636
"Жилое здание (Гостиница, Общежитие)",5,0.003030
Производственный объект,4,0.002424


In [13]:
cond = uninvoiced_objects["Тип Объекта"] == "Многоквартирный дом"
s = uninvoiced_objects[cond]['Общая площадь объекта'].sum() / 1e06

print(f"У {cond.sum()} объектов типа Многоквартирный дом общей площадью{s: 0.1f} млн. нет данных о выствленных счетах.")

У 752 объектов типа Многоквартирный дом общей площадью 1.1 млн. нет данных о выствленных счетах.


* У 1650 объектов нет данных об учете потребления теплоэнергии. Из них 752 объекта (46%) общей площадью 1.1 млн. приходится на тип ```Многоквартирный дом```, около 40% - на тип ```Другое строение```, около 8.3% - на тип ```Частный дом```.


#### 3.1.3 Неуникальные адреса объектов.

In [14]:
nonunique = buildings[
    buildings.duplicated(subset=["Адрес объекта", "Тип Объекта"], keep=False)
]
print(f"Всего неуникальных объектов: {nonunique.shape[0]}\n")
nonunique.head(6)

Всего неуникальных объектов: 145



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2
122,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,1,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
123,"г Уфа, б-р. Тухвата Янаби, д.34",Другое строение,2,NaT,0.01,"г Уфа, б-р. Тухвата Янаби, д.34"
412,"г Уфа, пр-кт Октября, д.153",Другое строение,0,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
413,"г Уфа, пр-кт Октября, д.153",Другое строение,3,NaT,0.01,"г Уфа, пр-кт Октября, д.153"
487,"г Уфа, пр-кт Октября, д.31",Другое строение,0,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"
488,"г Уфа, пр-кт Октября, д.31",Другое строение,5,1970-01-01,0.01,"г Уфа, пр-кт Октября, д.31"


In [15]:
pd.concat(
    [
      nonunique["Тип Объекта"].value_counts(),
      nonunique["Тип Объекта"].value_counts(True),
      nonunique.groupby(["Тип Объекта"])["Адрес объекта"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта
Тип Объекта,,,
Другое строение,116,0.800000,51
Многоквартирный дом,14,0.096552,7
Частный дом,12,0.082759,4
"Учебное заведение, комбинат, центр",3,0.020690,1


1. Типы объектов ```Другое строение``` и ```Частный дом``` оставлены без корректировок:
  * Уникальный адрес объекта нужен для привязки площади объекта и даты постройки к данным учета потребления теплоэнергии. У типа ```Другое строение``` - в большинстве случаев эти данные отстутствуют, соответственно этот блок информации не имеет смысла корректировать.

  * Информации об неуникальных объектах типа ```Частный дом``` нет в данных о потреблении теплоэнергии - этот блок также не имеет смысла корректировать.

2. Блок ```Многоквартирный дом``` скорректирован:
  * Адреса многоквартирных домов сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии.
  * На трех объектах ниже счета не выставляются, для них в признаке ```Адрес объекта 2``` к адресу добавлено слово ```extra``` :
    * ```г Уфа, ул. Вологодская, д.20```,
    * ```г Уфа, ул. Интернациональная, д.113```,
    * ```г Уфа, ул. Нежинская, д.6```.
3. Блок ```Учебное заведение, комбинат, центр``` скорректирован:
  * Адреса сделаны уникальными в признаке ```Адрес объекта 2``` через сопоставление с данными об учете теплоэнергии и проставление ``№ ОДПУ`` в адрес.

In [16]:
nonunique_2 = buildings[
    buildings.duplicated(subset=["Адрес объекта 2", "Тип Объекта"], keep=False)
]
pd.concat(
    [
      nonunique_2["Тип Объекта"].value_counts(),
      nonunique_2["Тип Объекта"].value_counts(True),
      nonunique_2.groupby(["Тип Объекта"])["Адрес объекта 2"].agg(lambda x: x.nunique())
    ], axis=1
)

,count,proportion,Адрес объекта 2
Тип Объекта,,,
Другое строение,115,0.905512,51
Частный дом,12,0.094488,4


* Если использовать ```Адрес объекта 2```, неуникальные адреса в типах  ```Многоквартирный дом``` и ```Учебное заведение, комбинат, центр``` появляться не будут.

## 3.2 Равные значения показаний в течение нескольких расчетных периодов

In [17]:
save = True
equal_values = get_equal_values(data, save, path=PATH)

print(f"Число записей с равными значениями показаний: {equal_values.shape[0]}")
print(f"Число адресов с равными значениями показаний: {equal_values['Адрес объекта'].nunique()}\n")
equal_values.head(10)

Число записей с равными значениями показаний: 248
Число адресов с равными значениями показаний: 97



,index,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал",Период потребления
36031,3465.0,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-01-01,188.0,2021-12-01
50282,693.0,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-07-01,21.0,2022-06-01
48479,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-09-01,12.0,2022-08-01
59404,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-10-01,12.0,2022-09-01
0,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2023-02-01,188.0,2023-01-01
19672,NaN,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2023-07-01,21.0,2023-06-01
58934,3904.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2022-04-01,70.0,2022-03-01
42679,47.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2022-12-01,70.0,2022-11-01
47078,27.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2023-06-01,2.0,2023-05-01
19697,26.0,Уфа,27834,ГВС-ИТП,"г Уфа, б-р. Ибрагимова, д.61",Другое строение,2023-07-01,2.0,2023-06-01


## 3.3. Аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами

(только для типов объекта «Многоквартирный дом») по критериям:
  - год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
  - этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
  - площадь (±10%),
  - наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).

In [18]:
grouping = Grouping()
df_grouped, dropped = grouping.fit_transform(df, buildings)

print(f"df_grouped.shape = {df_grouped.shape}\n")
df_grouped.sample(2)

df_grouped.shape = (3236, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
561,"г Уфа, ул. 50-летия Октября, д.2",Многоквартирный дом,9.0,1966-01-01,5831.0,"г Уфа, ул. 50-летия Октября, д.2",5-9 этажей,ул. 50-летия Октября,1966-01-01,1959-1989 гг.,...,NaN,8.043,19.907,30.397,32.97,28.292,25.877,14.512,NaN,NaN
821,"г Уфа, ул. Баязита Бикбая, д.32",Многоквартирный дом,12.0,1985-01-01,3866.9,"г Уфа, ул. Баязита Бикбая, д.32",10-12 этажей,ул. Баязита Бикбая,1985-01-01,1959-1989 гг.,...,NaN,0.000,14.400,26.560,25.09,NaN,14.560,12.200,NaN,NaN


In [19]:
dropped

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1444,"г Уфа, ул. Кирова, д.95",Многоквартирный дом,1.0,1971-01-01,0.01,"г Уфа, ул. Кирова, д.95, Подобъект №46590",1-2 этажа,ул. Кирова,1971-01-01,1959-1989 гг.,...,NaN,6.623,18.004,14.127,29.448,25.116,18.209,11.972,NaN,NaN
2686,"г Уфа, ул. Рудольфа Нуреева, д.1",Многоквартирный дом,20.0,2019-11-01,0.04,"г Уфа, ул. Рудольфа Нуреева, д.1",13 и более этажей,ул. Рудольфа Нуреева,2019-11-01,2011-2024 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Удалены объекты с неуказанной общей площадью:

1. Нет информации ```Этажность объекта```, ```Дата постройки```, ```Общая площадь объекта``` по многоквартирному дому по адресу ```г Уфа, ул. Уфимское шоссе, д.4```.

2. По адресу ```г Уфа, ул. Кирова, д.95``` значатся 2 многоквартирных дома: 5 этажей и 1 этаж. По одноэтажному дому (```Подобъект №46590```) нет информации по общей площади объекта.

3. Нет информации по общей площади объекта по адресу ```г Уфа, ул. Рудольфа Нуреева, д.1```.

In [20]:
save=True

n_periods = 24
threshold = 0.25

under_medians, over_medians, underconsumption, overconsumption = (
    get_outlers(df_grouped, threshold=threshold, n_periods=n_periods,  save=save, path=PATH)
)
print(f"Aномально низкое/высокое (отклонение более {threshold:.0%}) потребление:")
for period in under_medians:
  print(f"{period: %Y-%m}:  {len(under_medians[period])}\t{len(over_medians[period])}")

Aномально низкое/высокое (отклонение более 25%) потребление:
 2021-07:  127	148
 2021-08:  139	147
 2021-09:  217	217
 2021-10:  107	134
 2021-11:  88	90
 2021-12:  80	73
 2022-01:  82	59
 2022-02:  83	68
 2022-03:  88	77
 2022-04:  117	130
 2022-05:  126	159
 2022-06:  117	146
 2022-07:  136	163
 2022-08:  162	181
 2022-09:  144	146
 2022-10:  127	166
 2022-11:  81	92
 2022-12:  65	75
 2023-01:  66	80
 2023-02:  72	71
 2023-03:  83	84
 2023-04:  145	141
 2023-05:  118	155
 2023-06:  134	166


In [21]:
print(f"underconsumption.shape = {underconsumption.shape}\n")
underconsumption.sample(5)

underconsumption.shape = (639, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
1369,"г Уфа, ул. Калинина, д.8",Многоквартирный дом,4.0,1955-01-01,3170.6,"г Уфа, ул. Калинина, д.8",3-4 этажа,ул. Калинина,1955-01-01,до 1958 г,...,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.774,6.356
275,"г Уфа, пр-кт Октября, д.15 корп.1",Многоквартирный дом,9.0,1990-01-01,5774.3,"г Уфа, пр-кт Октября, д.15 корп.1",5-9 этажей,пр-кт Октября,1990-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.486,17.661,NaN
2053,"г Уфа, ул. Набережная р. Уфы, д.11/2Б",Многоквартирный дом,10.0,2014-01-01,5812.0,"г Уфа, ул. Набережная р. Уфы, д.11/2Б",10-12 этажей,ул. Набережная р. Уфы,2014-01-01,2011-2024 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.163
2533,"г Уфа, ул. Рихарда Зорге, д.54",Многоквартирный дом,9.0,1970-01-01,13692.4,"г Уфа, ул. Рихарда Зорге, д.54",5-9 этажей,ул. Рихарда Зорге,1970-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2122,"г Уфа, ул. Нежинская, д.17",Многоквартирный дом,4.0,1959-01-01,3630.6,"г Уфа, ул. Нежинская, д.17",3-4 этажа,ул. Нежинская,1959-01-01,до 1958 г,...,7.5381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
print(f"overconsumption.shape = {overconsumption.shape}\n")
overconsumption.sample(5)

overconsumption.shape = (524, 35)



,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта,Адрес объекта 2,Группа этажность объекта,Улица,Дата постройки 2,Группа год постройки,...,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00
2215,"г Уфа, ул. Пархоменко, д.104 корп.1",Многоквартирный дом,5.0,1973-01-01,2798.61,"г Уфа, ул. Пархоменко, д.104 корп.1",5-9 этажей,ул. Пархоменко,1973-01-01,1959-1989 гг.,...,NaN,69.528,103.141,140.781,148.589,130.435,106.302,95.479,NaN,NaN
584,"г Уфа, ул. 8 Марта, д.18",Многоквартирный дом,5.0,1969-01-01,3572.10,"г Уфа, ул. 8 Марта, д.18",5-9 этажей,ул. 8 Марта,1969-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,"г Уфа, ул. Братьев Кадомцевых, д.16",Многоквартирный дом,5.0,1966-01-01,3553.20,"г Уфа, ул. Братьев Кадомцевых, д.16",5-9 этажей,ул. Братьев Кадомцевых,1966-01-01,1959-1989 гг.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.235
2040,"г Уфа, ул. Мустая Карима, д.42",Многоквартирный дом,6.0,2005-01-01,4937.60,"г Уфа, ул. Мустая Карима, д.42",5-9 этажей,ул. Мустая Карима,2005-01-01,2001-2010 гг.,...,NaN,NaN,NaN,NaN,187.873,NaN,129.661,NaN,NaN,NaN
1594,"г Уфа, ул. Комсомольская, д.133",Многоквартирный дом,5.0,1963-01-01,2100.40,"г Уфа, ул. Комсомольская, д.133",5-9 этажей,ул. Комсомольская,1963-01-01,1959-1989 гг.,...,15.622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.026,16.932
